In [1]:
import csv

import os
import sys
from docplex.mp.progress import *
from docplex.mp.progress import SolutionRecorder
import networkx as nx
import time
from absl import flags
FLAGS = flags.FLAGS

In [28]:
def CPLEX_swap_scheduling():
    import docplex.mp.model as cpx
    opt_model = cpx.Model(name="swap_scheduling")
    eflow_vars  = {(i,j,k,b): opt_model.continuous_var(lb=0, ub= max_edge_capacity,
                    name="eflow_{0}_{1}_{2}_{3}".format(i,j,k,b))
                   for i in nodes for j in nodes for k in nodes for b in nodes}

    u_vars  = {(i,j): opt_model.continuous_var(lb=0, ub= max_edge_capacity,
                              name="u_{0}_{1}".format(i,j)) for i in nodes for j in nodes}
    
    for i in nodes:
        for j in nodes:
            if i!=j:
                if (i,j) != (source,sink) and (i,j)!=(sink,source):
                    opt_model.add_constraint(opt_model.sum(q_value[k] *
                    (eflow_vars[i,k,i,j]+eflow_vars[k,j,i,j])/2 for k in nodes if k not in [i,j])+
                    u_vars[i,j]*edge_capacity[(i,j)] * check_edge_exit(i,j)==opt_model.sum(
                    (eflow_vars[i,j,i,k]+eflow_vars[i,j,k,j]) for k in nodes if k not in [i,j]))
    for i in nodes:
        for j in nodes:
            for k in nodes:
                opt_model.add_constraint(eflow_vars[i,k,i,j]==eflow_vars[k,j,i,j])
                opt_model.add_constraint(eflow_vars[i,k,i,j]>=0)
    for i in nodes:
        opt_model.add_constraint(eflow_vars[source,sink,source,i]==eflow_vars[source,sink,sink,i])
        opt_model.add_constraint(eflow_vars[source,sink,source,i]==0)

                                             
            
    objective = opt_model.sum(q_value[k] * (eflow_vars[source,k,source,sink]+eflow_vars[k,sink,source,sink])/2
                                             for k in nodes if k not in [source,sink]
                          )

    
    # for maximization
    opt_model.maximize(objective)
#     opt_model.solve()
    opt_model.print_information()
    #try:
    opt_model.solve()
    print('docplex.mp.solution',opt_model.solution)
    
    objective_value = -1
    try:
        if opt_model.solution:
            objective_value =opt_model.solution.get_objective_value()
    except ValueError:
        print(ValueError)
        
    print("EGR is ",objective_value)
    
#     for i in nodes:
#         for j in nodes:
#             for k in nodes:
#                 print("eflow_vars(%s,%s,%s,%s) %s"%(i,k,i,j,eflow_vars[i,k,i,j].solution_value))
#                 print("eflow_vars(%s,%s,%s,%s) %s"%(k,j,i,j,eflow_vars[k,j,i,j].solution_value))

In [29]:
def check_edge_exit(i,j):
    if (i,j) in edges:
        return 1
    else:
        return 0
max_edge_capacity=100
edges = [(1,2),(2,3),(3,4),(4,5)]
nodes = [1,2,3,4,5]
source =1
sink = 5
edge_capacity = {}
q_value = {}
for node in nodes:
    q_value[node] =1
    

for edge in edges:
    edge_capacity[edge] = 100
for i in nodes:
    for j in nodes:
        if (i,j) not in edges:
            edge_capacity[(i,j)] = 0
CPLEX_swap_scheduling()

Model: swap_scheduling
 - number of variables: 650
   - binary=0, integer=0, continuous=650
 - number of constraints: 278
   - linear=278
 - parameters: defaults
 - objective: maximize
 - problem type is: LP
docplex.mp.solution solution for: swap_scheduling
objective: 300
eflow_1_2_1_3=100.000
eflow_1_2_1_4=100.000
eflow_1_2_1_5=100.000
eflow_1_3_1_5=100.000
eflow_1_4_1_5=100.000
eflow_1_5_3_5=100.000
eflow_2_3_1_3=100.000
eflow_2_3_2_4=100.000
eflow_2_3_2_5=100.000
eflow_2_4_1_4=100.000
eflow_2_5_1_5=100.000
eflow_3_1_3_5=100.000
eflow_3_4_2_4=100.000
eflow_3_4_3_1=100.000
eflow_3_4_3_5=100.000
eflow_3_5_1_5=100.000
eflow_3_5_2_5=100.000
eflow_4_1_3_1=100.000
eflow_4_5_1_5=100.000
eflow_4_5_3_5=100.000
eflow_4_5_4_1=100.000
eflow_5_1_4_1=100.000
u_1_2=3.000
u_2_3=3.000
u_3_4=3.000
u_4_5=3.000

EGR is  300.0
